# Clean

This notebook loads your assembled data and prepares it for modeling.

The final output is a `sup` that is fully ready for modeling and analysis.

Here's what happens in this notebook:
- We fill gaps in the data using reasonable assumptions
- We prepare the data for further analysis by marking clusters
- We process sales validity information
- We run our own sales scrutiny heuristic to make sure we only use trustworthy sales that reflect market value

These operations are necessary for modeling, but nevertheless inject a certain amount of subjectivity into the model, which is why we keep the results of the **Clean** notebook separate from those produced in the **Assemble** notebook.

In [1]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-md-baltimorecity"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = True

# 1. Basic setup

In [2]:
import init_notebooks
init_notebooks.setup_environment()

Environment setup completed.


In [3]:
# import a bunch of stuff
from openavmkit.pipeline import (
    init_notebook,
    from_checkpoint,
    delete_checkpoints,
    write_checkpoint,
    read_pickle,
    load_settings,
    examine_sup,
    fill_unknown_values_sup,
    process_sales,
    mark_ss_ids_per_model_group_sup,
    mark_horizontal_equity_clusters_per_model_group_sup,
    run_sales_scrutiny_per_model_group_sup,
    write_notebook_output_sup
)

In [4]:
init_notebook(locality)

locality = us-md-baltimorecity
base path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline
current path = /Users/gregmiller/Documents/openavmkit/openavmkit/notebooks/pipeline/data/us-md-baltimorecity


In [5]:
if clear_checkpoints:
    delete_checkpoints("2-clean")

# 2. Load data

In [6]:
settings = load_settings()

In [7]:
# load the data
sales_univ_pair = read_pickle("out/1-assemble-sup")

In [8]:
examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
exempt_land_value               Float64       16,452    7%    236,471  100%                                         
geom_aspect_ratio               float64      236,505  100%    236,505  100%                    

# 3. Fill unknowns

Modeling functions are unable to process null data, so you need to fill them in somehow.   
The goal is to **eliminate all gaps in your data,** at least for fields you intend to turn into modeling variables.

Consult the documentation for more details and best practices on filling unknown values.

In [9]:
# Fill holes in the data with sensible defaults
sales_univ_pair = fill_unknown_values_sup(sales_univ_pair, settings)

/Users/gregmiller/Documents/openavmkit/openavmkit/openavmkit/cleaning.py:369: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df[field].fillna(False)
/Users/gregmiller/Documents/openavmkit/openavmkit/openavmkit/cleaning.py:369: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = df[field].fillna(False)
/Users/gregmiller/Documents/openavmkit/openavmkit/openavmkit/cleaning.py:369: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) inste

In [10]:
# Look at the data and make sure it makes sense
examine_sup(sales_univ_pair, settings)


EXAMINING UNIVERSE...

            FIELD                 TYPE     NON-ZERO    %    NON-NULL    %                    UNIQUE                 
============================== ========== ========== ===== ========== ===== ========================================
             LAND             
============================== ========== ========== ===== ========== ===== ========================================
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
           NUMERIC            
------------------------------ ---------- ---------- ----- ---------- ----- ----------------------------------------
assr_land_value                 Float64      219,315   93%    236,505  100%                                         
exempt_land_value               Float64       16,452    7%    236,471  100%                                         
geom_aspect_ratio               float64      236,505  100%    236,505  100%                    

# 4. Clustering

We cluster all similar properties and give each cluster a unique ID.  
Later, we'll use these ID's whenever we want to run a horizontal equity study.


In [11]:
settings = load_settings()
sales_univ_pair = from_checkpoint("2-clean-00-horizontal-equity", mark_horizontal_equity_clusters_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)


Marking horizontal equity clusters...
Processing model group: apartments
--> crunching on location, 153 clusters
--> crunching on is_vacant, 153 clusters
----> Processing batch 0-100 of 153
----> Processing batch 100-153 of 153
--> crunching on land_area_sqft, 153 clusters
----> Processing batch 0-100 of 153
----> Processing batch 100-153 of 153
--> crunching on bldg_area_finished_sqft, 165 clusters
----> Processing batch 0-100 of 165
----> Processing batch 100-165 of 165
--> crunching on bldg_quality_num, 173 clusters
----> Processing batch 0-100 of 173
----> Processing batch 100-173 of 173
--> crunching on bldg_condition_num, 173 clusters
----> Processing batch 0-100 of 173
----> Processing batch 100-173 of 173
--> crunching on bldg_age_years, 173 clusters
----> Processing batch 0-100 of 173
----> Processing batch 100-173 of 173
Processing model group: rowhome
--> crunching on location, 241 clusters
--> crunching on is_vacant, 241 clusters
----> Processing batch 0-100 of 241
----> 0

## 5. Process sales

We process sales validity information to set all the right codes for later use.  
We calculate time trends for sales over time to generate time-adjusted sale prices.

In [12]:
sales_univ_pair = from_checkpoint("2-clean-01-process_sales", process_sales,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)

Before univ merge len = 54124
After univ merge len = 54124
Using 54124 sales...
--> 599 vacant sales
--> 53525 improved sales
--> 32735 valid for ratio study
--> 599 valid for land ratio study
len before hydrate = 54124
len after hydrate = 54124
len after enrich = 54124


## 6. Scrutinize sales

We cluster all sales of similar properties in similar locations.  
We flag individual sales that are anomalously high or low for their local cluster.  
This helps us catch potentially invalid sales that slipped by the assessor's notice.  

In [13]:
sales_univ_pair = from_checkpoint("2-clean-02-mark-ss_ids", mark_ss_ids_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose,
    }
)

--> crunching on location, 61 clusters
--> crunching on land_area_sqft, 61 clusters
----> Processing batch 0-61 of 61
--> crunching on bldg_area_finished_sqft, 64 clusters
----> Processing batch 0-64 of 64
--> crunching on bldg_quality_num, 65 clusters
----> Processing batch 0-65 of 65
--> crunching on ['bldg_effective_age_years', 'bldg_age_years'], 65 clusters
----> Processing batch 0-65 of 65
--> crunching on bldg_condition_num, 65 clusters
----> Processing batch 0-65 of 65
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/gregmiller/Documents/openavmkit/openavmkit/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jb/s1bhbc2x3dgbhdvtbnz335sc0000gn/T/ipykernel_20022/2465474694.py", line 1, in <module>
    sales_univ_pair = from_checkpoint("2-clean-02-mark-ss_ids", mark_ss_ids_per_model_group_sup,
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gregmiller/Documents/openavmkit/openavmkit/openavmkit/pipeline.py", line 653, in from_checkpoint
    return openavmkit.checkpoint.from_checkpoint(path, func, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gregmiller/Documents/openavmkit/openavmkit/openavmkit/checkpoint.py", line 16, in from_checkpoint
    result = func(**params)
             ^^^^^^^^^^^^^^
  File "/Users/gregmiller/Documents/op

In [28]:
sales_univ_pair = from_checkpoint("2-clean-03-sales-scrutiny", run_sales_scrutiny_per_model_group_sup,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U2'), dtype('float64')) -> None

# 7. Write out results

In [ ]:
write_notebook_output_sup(sales_univ_pair, "2-clean")

# 8. Look at it on a map!
- Take the files output in the previous step and put them in a map viewer like QGIS, ArcGIS, or Felt
- Look at them with your eyeballs
- Make sure the data looks correct
- If not, go back and fix it!
- Don't proceed to the next step until everything looks right